In [1]:

from bs4 import BeautifulSoup
import sqlite3
import requests
import pandas as pd
from random import randint
import time
from time import sleep
from typing import List
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model, metrics, preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import r2_score, f1_score

In [6]:
def sleepms(milliseconds: int):
    seconds = 0.001 * milliseconds
    sleep(seconds)
    
def random_wait():
    start = time.time()
    sleepms(randint(1, 250))

def get_fake_user_agent():
    return {"User-Agent":"PostmanRuntime/7.29.0"}


def getRetHeaders():
    url='https://www.ret.co.il/m/91935/specs'
    page = requests.get(url, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    product=soup.find('div',class_='single-product-tab')
    data=product.findAll('ul','list-group list-group-horizontal')
    myheaders=[]
    for alldata in data:
        myheaders.append(alldata.find('span',class_='text-dark font-weight-bold').text.strip())
    return myheaders



def getUrl(url):
    page = requests.get(url, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    MainClass=soup.find('ul', class_='nav nav-pills mb-2 justify-content-around justify-content-md-start col-xxl-10 offset-xxl-1 col-md-12 offset-md-0')
    ref=[]
    data=MainClass.find('a')
    technicalDataPage='https://www.ret.co.il'+data.get('href')
    return technicalDataPage

def getRetData(url):
    technicalDataPage=getUrl(url)
    page = requests.get(technicalDataPage, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    product=soup.find('div',class_='single-product-tab')
    dataLink=product.findAll('ul',class_='list-group list-group-horizontal')
    ref=[]
    mydata=[]
    myheaders=[]
    for alldata in dataLink:
        try:
            mydata.append(alldata.find('li',class_='list-group-item col-8 rounded-0 text-600').text.strip())
        except:
            mydata.append('None')
    return mydata



In [7]:
def getRetToDataFrame(phonesURL):
    hrefLinks=[]
    allRetData=[]
    page = requests.get(phonesURL, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    MainClass=soup.findAll('a')
    for href in MainClass:
        if('/m/' in href.get('href')):
            hrefLinks.append('https://www.ret.co.il' + href.get('href'))
    for hrefLink in hrefLinks:
        allRetData.append(getRetData(hrefLink))
    return allRetData


In [50]:
def mainFunc(page_num):
    head=getRetHeaders()
    url=f"https://www.ret.co.il/shop/cellphone?pageindex={page_num}"
    data=getRetToDataFrame(url)
    df=pd.DataFrame(data,columns=head)
    return df
    # headers=getRetHeaders()
    # AllData=[]

    #    AllData.append(getRetToDataFrame(url))
    #    random_wait()    
    # df=pd.DataFrame(list(zip(AllData),columns=headers))
    # return df

In [56]:
df=mainFunc(1)
for i in range(2,10):
    df=df.append(mainFunc(i))
    print(df.shape[0])
    random_wait()
df

C:\Users\david\AppData\Local\Temp\ipykernel_13604\1775400901.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(mainFunc(i))


96


C:\Users\david\AppData\Local\Temp\ipykernel_13604\1775400901.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(mainFunc(i))


144


C:\Users\david\AppData\Local\Temp\ipykernel_13604\1775400901.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(mainFunc(i))


...

C:\Users\david\AppData\Local\Temp\ipykernel_13604\1775400901.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(mainFunc(i))


384


C:\Users\david\AppData\Local\Temp\ipykernel_13604\1775400901.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(mainFunc(i))


406


,יצרן,מערכת הפעלה,שנת הכרזה,מבנה המכשיר,סדרה,Smartphone,רמת קרינה (SAR),ממשק הפעלה,צבעים,טלפון כשר,...,חיבור 3.5,רמקול חיצוני,רדיו FM,גובה,רוחב,עובי,משקל,קיבולת הסוללה,סוללה נשלפת,טעינה אלחוטית
0,Apple,iOS,2020,פאבלט,iPhone 12,סמארטפון,None,מסך מגע,None,None,...,ללא,זוג רמקולים,ללא רדיו,"146.1 מ""מ","71.5 מ""מ","7.4 מ""מ",164 גרם,None,לא,כולל
1,Apple,iOS,2019,פאבלט,iPhone 11,סמארטפון,None,מסך מגע,None,None,...,ללא,זוג רמקולים,ללא רדיו,"150.9 מ""מ","75.7 מ""מ","8.3 מ""מ",194 גרם,"3,110 mAh",לא,כולל
2,Samsung,None,2021,פאבלט,Galaxy S21,סמארטפון,0.71 W/kg,מסך מגע,None,None,...,ללא,זוג רמקולים,ללא רדיו,"165.1 מ""מ","75.6 מ""מ","8.9 מ""מ",227 גרם,"5,000 mAh",לא,כולל
3,Samsung,Android,2021,פאבלט,Galaxy S21,סמארטפון,0.46 W/kg,מסך מגע,None,None,...,ללא,זוג רמקולים,ללא רדיו,"151.7 מ""מ","71.2 מ""מ","7.9 מ""מ",169 גרם,"4,000 mAh",לא,כולל
4,Apple,iOS,2020,פאבלט,iPhone 12 Pro Max,סמארטפון,None,מסך מגע,None,None,...,ללא,זוג רמקולים,ללא רדיו,"160.8 מ""מ","78.1 מ""מ","7.4 מ""מ",228 גרם,None,לא,כולל
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,Samsung,Android,None,פאבלט,None,סמארטפון,None,מסך מגע,None,None,...,ללא,יחיד,None,"163.7 מ""מ","76.1 מ""מ","7.6 מ""מ",181 גרם,"5,000 mAh",לא,None
18,Samsung,Android,2021,פאבלט,Galaxy A52,סמארטפון,1.05 W/kg,מסך מגע,None,None,...,כולל,זוג רמקולים,כולל רדיו,"159.9 מ""מ","75.1 מ""מ","8.4 מ""מ",189 גרם,"4,500 mAh",לא,None
19,Samsung,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
20,Xiaomi,Android,None,פאבלט,None,סמארטפון,None,מסך מגע,None,None,...,None,זוג רמקולים,None,"164.2 מ""מ","76.1 מ""מ","8.1 מ""מ",202 גרם,"5,000 mAh",לא,ללא


In [66]:
def get_data_from_zap(phonesURL):
    hrefLinks=[]
    allRetData=[]
    page = requests.get(phonesURL, headers = get_fake_user_agent())
    soup= BeautifulSoup(page.content,'html.parser')
    links=soup.find_all('div',class_="MoreInfo")
    for link in links:
        print(link.find('a')['href'])

In [67]:
def get_details_from_current_page(page_num):
    url="https://www.zap.co.il/models.aspx?sog=e-cellphone&pageinfo={page_num}"
    page=data=get_data_from_zap(url)


In [68]:
get_details_from_current_page("https://www.zap.co.il/models.aspx?sog=e-cellphone&pageinfo=1")

/compmodels.aspx?modelid=1142728
/compmodels.aspx?modelid=1168574
/compmodels.aspx?modelid=1149678
/compmodels.aspx?modelid=1124638
/compmodels.aspx?modelid=1168584
/compmodels.aspx?modelid=1142731
/compmodels.aspx?modelid=1078896
/compmodels.aspx?modelid=1137460
/compmodels.aspx?modelid=1168578
/compmodels.aspx?modelid=1146138
/compmodels.aspx?modelid=1140819
/compmodels.aspx?modelid=1142730
/compmodels.aspx?modelid=1168577
/compmodels.aspx?modelid=1146137
/compmodels.aspx?modelid=1145966
/compmodels.aspx?modelid=1148409
/compmodels.aspx?modelid=1142729
/compmodels.aspx?modelid=1091548
/compmodels.aspx?modelid=1124722
/compmodels.aspx?modelid=1124639
/compmodels.aspx?modelid=1171258
/compmodels.aspx?modelid=1040282
/compmodels.aspx?modelid=1168573
/compmodels.aspx?modelid=1163391
